# 闭包

* **闭包在装饰器中很有用，也是回调式异步编程和函数式编程的基础**      
* **闭包指延伸了作用域的函数，能访问定义体之外定义的非全局变量**
* **只有设计嵌套函数时才有闭包问题**

In [2]:
# 计算不断增加的系列值的均值
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

avg = make_averager()

**在averager函数中，series是自由变量，指未在本地作用域中绑定的变量**       
**其实针对嵌套内层函数的自由变量，是嵌套外层函数的局部变量**

In [3]:
avg(10)

10.0

In [4]:
avg(11)

10.5

In [5]:
avg(12)

11.0

**series的绑定在返回的avg函数的__closure__属性中，其中的各个元素对应于__code__.co_freevars中的一个名称，这些元素是cell对象，它的cell_contents属性保存着series真正的值**

In [6]:
avg.__code__.co_varnames

('new_value', 'total')

In [7]:
avg.__code__.co_freevars

('series',)

In [8]:
avg.__closure__

(<cell at 0x000001BAC647FFA8: list object at 0x000001BAC64F7108>,)

In [11]:
avg.__closure__[0].cell_contents

[10, 11, 12]

### 总结：
* **闭包是一种函数，保留定义函数时存在的自由变量的绑定，这样调用函数时虽然定义作用域不可用，但是仍能使用那些绑定**
* **只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量**

# nonlocal声明

In [12]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

**出错是因为，当count是数字或任何不可变类型时，count += 1 实际上与 count = count + 1 一样，在averager定义体中进行了赋值，会把count作为局部变量；total也一样**

**上面的series是列表，没有给他赋值，只是调用了它的方法，所以没有问题，这也是利用了列表是可变对象这一点**

**Python3引入nonlocal声明，把变量标记为自由变量**

In [13]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

10.0